In [311]:
import cv2
import mediapipe as mp
import time
import numpy as np
from PIL import Image, ImageDraw, ImageFilter
import image_similarity_measures
from image_similarity_measures.quality_metrics import ssim
from colormath.color_objects import sRGBColor, LabColor
from colormath.color_conversions import convert_color
from colormath.color_diff import delta_e_cie2000
import sys, os
import scipy
import scipy.misc
import scipy.cluster
import binascii
import struct

#cap = cv2.VideoCapture("D:\\Face\\MediaPipe\\output_25fps.mp4") #"Videos/1.mp4"
pTime = 0

def get_dominant_colour(img_path):
    try:
        NUM_CLUSTERS = 5
        im = Image.open(img_path)
        #im = im.resize((150, 150))      # optional, to reduce time
        ar = np.asarray(im)
        shape = ar.shape
        ar = ar.reshape(scipy.product(shape[:2]), shape[2]).astype(float)
        codes, dist = scipy.cluster.vq.kmeans(ar, NUM_CLUSTERS)
        vecs, dist = scipy.cluster.vq.vq(ar, codes)         # assign codes
        counts, bins = scipy.histogram(vecs, len(codes))    # count occurrences    
        index_max = scipy.argmax(counts)                    # find most frequent
        peak = codes[index_max]
        colour = binascii.hexlify(bytearray(int(c) for c in peak)).decode('ascii')
        r = int(peak[0])
        g = int(peak[1])
        b = int(peak[2])
        bgr = (b, g, r)
        return bgr
    except:
        pass
    return (-1,-1,-1)

def compare_color(bgr1, bgr2):
    # Red Color
    color1_rgb = sRGBColor(bgr1[2],bgr1[1], bgr1[0]);

    # Blue Color
    color2_rgb = sRGBColor(bgr2[2],bgr2[1], bgr2[0]);

    # Convert from RGB to Lab Color Space
    color1_lab = convert_color(color1_rgb, LabColor);

    # Convert from RGB to Lab Color Space
    color2_lab = convert_color(color2_rgb, LabColor);

    # Find the color difference
    delta_e = delta_e_cie2000(color1_lab, color2_lab);

#     print ("The difference between the 2 color = ", delta_e)
    return round(delta_e, 2)
    
def compare_SSIM(img1,img2):
#     print(img1,img2)
    in_img1 = cv2.imread(img1)
    in_img2 = cv2.imread(img2)

    img1_outputimage = 'temp1.jpg'
    img2_outputimage = 'temp2.jpg'

    temp1_image = cv2.resize(in_img1, (100, 50), interpolation=cv2.INTER_NEAREST)
    cv2.imwrite(img1_outputimage, temp1_image)

    temp2_image = cv2.resize(in_img2, (100, 50), interpolation=cv2.INTER_NEAREST)
    cv2.imwrite(img2_outputimage, temp2_image)

    out_ssim = ssim(temp1_image, temp2_image) # Structural Similar Index Measure (SSIM)
#     print('SSIM : ', out_ssim)
    return round(out_ssim,2)

def AngleBtw2Points(pointA, pointB):
    changeInX = pointB[0] - pointA[0]
    changeInY = pointB[1] - pointA[1]
    return math.degrees(math.atan2(changeInY,changeInX))

imgs, palette, thres, Hor_angle, Ver_angle, left_eye_angle, left_eye_Hi, Right_eye_angle, Right_eye_Hi, Lashes = [], [], [], [], [], [], [], [], [], []

images = 30
for nu in range(1,images+1):
# for nu in [30]:
    imgPath = f"D:\\Face\lips\\images\\{nu}.jpg"
    print("Input:", imgPath)
    paletteImages = "red-coat.png,ruby-rush.png,red-rust.png"
    temp_image = r"temp.jpg"
    eye_le = Image.open('lashes.png')
    eye_ri = Image.open('lashes1.png')
#     eye_ri = eye_up.transpose(Image.FLIP_LEFT_RIGHT)
    
    mpDraw = mp.solutions.drawing_utils
    mpFaceMesh = mp.solutions.face_mesh
    faceMesh = mpFaceMesh.FaceMesh(max_num_faces=2)
    drawSpec = mpDraw.DrawingSpec(thickness=1, circle_radius=2)

    face = [10, 338, 297, 332, 284, 251, 389, 356, 454, 323, 361, 288, 397, 365, 379, 378, 400, 377, 152, 
            148, 176, 149, 150, 136, 172, 58, 132, 93, 234, 127, 162, 21, 54, 103, 67, 109, 10, 168,
            133, 173, 157, 158, 159, 160, 161, 246, 22, 7, 163, 144, 145, 153, 154, 155, 133, 168,
            362, 398, 384, 385, 386, 387, 388, 466, 263, 249, 390, 373, 374, 380, 381, 382, 362, 168,
            0, 267, 269, 270, 409, 291, 375, 321, 405, 314, 17, 84, 181, 91, 146, 61, 185, 40, 39, 37, 0, 168]
    
    lips = [61, 185, 40, 39, 37, 0, 267, 269, 270, 409, 291,
            375, 321, 405, 314, 17, 84, 181, 91, 146, 61,
            78, 191, 80, 81, 82, 13, 312, 311, 310, 415, 308,
            324, 318, 402, 317, 14, 87, 178, 88, 95, 78]

    lips_up = [61, 185, 40, 39, 37, 0, 267, 269, 270, 409, 291]
    lips_down = [375, 321, 405, 314, 17, 84, 181, 91, 146, 61]


    l_eye = [33, 161, 160, 159, 158, 157, 173, 133, 155, 154, 153, 145, 144, 163]
    r_eye = [362, 398, 384, 385, 386, 387, 388, 466, 263, 249, 390, 373, 374, 380, 381, 382]
    
    #while True:
        #success, img = cap.read()
    img = cv2.imread(imgPath)
    ih, iw, ic = img.shape
    overlay = img.copy()

    frame = cv2.imread(imgPath)
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    lower_red = np.array([30,150,50])
    upper_red = np.array([255,255,180])
    mask = cv2.inRange(hsv, lower_red, upper_red)
    res = cv2.bitwise_and(frame,frame, mask= mask)
    edges = cv2.Canny(frame,50,100)
    cv2.imwrite(f'edge{nu}.jpg', edges)
    edg = cv2.imread(f'edge{nu}.jpg')

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    paletteImageList = paletteImages.split(',')
    index =1
    # print(paletteImageList)
    height = overlay.shape[0]
    width = overlay.shape[1]
    empty_mask = np.zeros((height, width), dtype=np.uint8)

    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # imgRGB = cv2.cvtColor(edg, cv2.COLOR_BGR2RGB)

    results = faceMesh.process(imgRGB)
    if results.multi_face_landmarks:
    #print(results.multi_face_landmarks)
        for faceLms in results.multi_face_landmarks:
            mpDraw.draw_landmarks(img, faceLms, mpFaceMesh.FACE_CONNECTIONS, drawSpec,drawSpec)

            points = []
            le_pts = []
            re_pts = []
            fpoints = []
            for id,lm in enumerate(lips):
                xx = faceLms.landmark[lm].x*iw
                yy = faceLms.landmark[lm].y*ih
                points.append((xx, yy)) 
                y_flag = yy
                if lm in lips_up:
                    points[id] = (xx, yy-yy*0.001)

                elif lm in lips_down:
                    points[id] = (xx, yy+yy*0.001)
                    
            for id,lm in enumerate(face):
                xx = faceLms.landmark[lm].x*iw
                yy = faceLms.landmark[lm].y*ih
                fpoints.append((xx, yy)) 
                y_flag = yy
                if lm in lips_up:
                    fpoints[id] = (xx, yy)
                    
            for id,lm in enumerate(l_eye):
                if lm in l_eye:
                    xx = int(faceLms.landmark[lm].x*iw)
                    yy = int(faceLms.landmark[lm].y*ih)                
                    le_pts.append((xx, yy))  
                    if lm == 33:
                        le_le = [xx, yy]
                    if lm == 133:
                        le_ri = [xx, yy]                    
                    if lm == 159:
                        le_up = [xx, yy]                    
                    if lm == 145:
                        le_do = [xx, yy]
                        
            for id,lm in enumerate(r_eye):
                if lm in r_eye:
                    xx = int(faceLms.landmark[lm].x*iw)
                    yy = int(faceLms.landmark[lm].y*ih) 
                    if lm == 362:
                        ri_le = [xx, yy]
                    if lm == 263:
                        ri_ri = [xx, yy]                    
                    if lm == 386:
                        ri_up = [xx, yy]                    
                    if lm == 374:
                        ri_do = [xx, yy]                        
                        
            h1 = [int(faceLms.landmark[33].x*iw), int(faceLms.landmark[33].y*ih)]
            h2 = [int(faceLms.landmark[263].x*iw), int(faceLms.landmark[263].y*ih)]
            v1 = [int(faceLms.landmark[5].x*iw), int(faceLms.landmark[5].y*ih)]
            v2 = [int(faceLms.landmark[0].x*iw), int(faceLms.landmark[0].y*ih)]
            lpt1 = [int(faceLms.landmark[70].x*iw), int(faceLms.landmark[70].y*ih)]
    print('-'*20)
    print("Face is Horizontally inclined by", abs(round(AngleBtw2Points(h1, h2),2)), 'degree')
    print("Face is Vertically inclined by", abs(round(90-AngleBtw2Points(v1, v2),2)), 'degree')
    print('-'*20)

    # points.append(points[0])    
    points = np.reshape(points, (-1, 1, 2))  
    points1 = np.int32([points])
    cv2.fillPoly(img, points1, (0, 0, 255), 8)         

    for paletteImage in paletteImageList:
        
        im_final = apply_makeup('cool-ivory.png', imgPath, fpoints, empty_mask)
        im_final.save(f"Out_{paletteImage}")
        
        imgPath = f"Out_{paletteImage}"
                                
        imgs.append(imgPath)
        palette.append(paletteImage)    
            
    #     print(paletteImage)
        #imgOutPath = outputFolder + str(index) + '.jpg'
        bgr_p = get_dominant_colour(paletteImage)
    #     print(bgr_p)

        #cv2.imwrite('raw.jpg',img)

        cv2.fillPoly(empty_mask, points1, (255))
        res = cv2.bitwise_and(overlay,overlay,mask = empty_mask)

        background = cv2.imread(imgPath)
        overlay = cv2.imread(paletteImage)
        overlay= cv2.resize(overlay, (background.shape[1], background.shape[0]))

        rect = cv2.boundingRect(points.astype(np.int))# returns (x,y,w,h) of the rect
        cropped = background[rect[1]-20: rect[1] + rect[3]+20, rect[0]-10: rect[0] + rect[2]+10]
        cv2.imwrite('temp2.jpg', cropped)

        sharpen_filter = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
        background = cv2.filter2D(background, -1, sharpen_filter)

        color_control = []
        color_SSIM = []
        color_Labspace = []
        for i in range(2,8,1):
            added_image = cv2.addWeighted(background,i/10,overlay,0.6,0)
            time.sleep(1)
            cv2.imwrite(temp_image, added_image)

            orig = cv2.imread(temp_image)
            rect = cv2.boundingRect(points.astype(np.int)) # returns (x,y,w,h) of the rect
    #         print(rect[1], rect[1] + rect[3], rect[0], rect[0] + rect[2])
            cropped = orig[rect[1]-20: rect[1] + rect[3]+20, rect[0]-10: rect[0] + rect[2]+10]
            cv2.imwrite('temp1.jpg', cropped)

            bgr = get_dominant_colour('temp1.jpg')

            d=((bgr[0]-bgr_p[0])**2+(bgr[1]-bgr_p[1])**2+(bgr[2]-bgr_p[2])**2)**0.5
            p=d*100/((255)^2+(255)^2+(255)^2)**0.5

            color_control.append(i/10)
            color_SSIM.append(round(compare_SSIM('temp2.jpg','temp1.jpg'),2))
            color_Labspace.append(round(compare_color(bgr_p, bgr),2))
            print(nu, paletteImage, i/10 ,bgr, round(d,2), round(p,2), compare_color(bgr_p, bgr),compare_SSIM('temp2.jpg','temp1.jpg'))

        color_control1 = []
        color_SSIM1 = []
        color_Labspace1 = []
        for i, val in enumerate(color_SSIM):
            if val > 0.8:
                color_control1.append(color_control[i])
                color_SSIM1.append(color_SSIM[i])
                color_Labspace1.append(color_Labspace[i])
#                 print(color_Labspace1)

        added_image = cv2.addWeighted(background,color_control1[color_Labspace1.index(min(color_Labspace1))],overlay,0.6,0)
        cv2.imwrite(temp_image, added_image)
        thresh_val = color_control1[color_Labspace1.index(min(color_Labspace1))]
        print('Best match: ', thresh_val, min(color_Labspace1))

        thres.append(thresh_val)
        
        im2 = Image.open(imgPath)
        im1 = Image.open(temp_image)

        height = im2.size[1]
        width = im2.size[0]

        empty_mask = np.zeros((height, width), dtype=np.uint8)
        cv2.fillPoly(empty_mask, points1, (255,255,255))
        cv2.imwrite('mask.jpg', empty_mask)


        mask = Image.open('mask.jpg').convert('L').resize(im1.size)
        mask_blur = mask.filter(ImageFilter.GaussianBlur(3))
        im_final = Image.composite(im1, im2, mask_blur)
        
        #Left eye
        # Calculate Euclidean distance
        eye1_w = math.sqrt(((le_ri[0]-le_le[0])**2)+((le_ri[1]-le_le[1])**2))
        eye1_h = math.sqrt(((le_do[0]-le_up[0])**2)+((le_do[1]-le_up[1])**2))
        rot1 = AngleBtw2Points(le_le, le_ri)

        le_x_resize = int(eye1_w*1.4)
        le_y_resize = int(eye1_h*2.4)

        eye_le = eye_le.resize((le_x_resize,le_y_resize))
        eye_le = eye_le.rotate(rot1*-1+4, expand=True)

        #Right_eye
        eye2_w = math.sqrt(((ri_ri[0]-ri_le[0])**2)+((ri_ri[1]-ri_le[1])**2))
        eye2_h = math.sqrt(((ri_do[0]-ri_up[0])**2)+((ri_do[1]-ri_up[1])**2))
        rot2 = AngleBtw2Points(ri_le, ri_ri)

        ri_x_resize = int(eye2_w*1.4)
        ri_y_resize = int(eye2_h*2.4)
        
        eye_ri = eye_ri.resize((ri_x_resize,ri_y_resize))
        eye_ri = eye_ri.rotate(rot2*-1-4, expand=True)        
        print(rot1,rot2)
        
        #lashes
        width = 800
        height = 519
        Ymargin_le = 320*le_y_resize/height
        Xmargin_le = 220*le_x_resize/width
        Ymargin_ri = 370/(height/ri_y_resize)
        Xmargin_ri = -10/(width/ri_x_resize)

        im_final.paste(eye_le, (int(le_le[0]-Xmargin_le),int(le_le[1]-Ymargin_le)), mask = eye_le)
        im_final.paste(eye_ri, (int(ri_le[0]-Xmargin_ri),int(ri_le[1]-Ymargin_ri)), mask = eye_ri)
#         im_final.save(f"Out_{paletteImage}")
        

        im_final.save(f"{nu}_Out_{paletteImage}")

        Hor_angle.append(abs(round(AngleBtw2Points(h1, h2),2)))
        Ver_angle.append(abs(round(90-AngleBtw2Points(v1, v2),2)))
        left_eye_angle.append(abs(round(rot1,2)))
        left_eye_Hi.append(round(eye1_h))
        Right_eye_angle.append(abs(round(rot2,2)))
        Right_eye_Hi.append(round(eye2_h))
        
        if abs(rot1) > 8 or abs(rot2) > 8 or eye1_h < 10 or eye2_h < 10:
            Lashes.append('No')
        else:
            Lashes.append('Yes')
        
        index = index +1
        break
    os.remove(f'edge{nu}.jpg')

os.remove('temp.jpg')
os.remove('temp1.jpg')
os.remove('temp2.jpg')


Input: D:\Face\lips\images\1.jpg
--------------------
Face is Horizontally inclined by 1.01 degree
Face is Vertically inclined by 0.52 degree
--------------------


C:\Users\css120809\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: DeprecationWarning: scipy.product is deprecated and will be removed in SciPy 2.0.0, use numpy.product instead
C:\Users\css120809\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: DeprecationWarning: scipy.histogram is deprecated and will be removed in SciPy 2.0.0, use numpy.histogram instead
C:\Users\css120809\Anaconda3\lib\site-packages\ipykernel_launcher.py:32: DeprecationWarning: scipy.argmax is deprecated and will be removed in SciPy 2.0.0, use numpy.argmax instead


1 red-coat.png 0.2 (45, 16, 152) 67.18 422.35 21.03 0.77
1 red-coat.png 0.3 (52, 22, 163) 57.58 361.98 18.22 0.84
1 red-coat.png 0.4 (59, 29, 174) 50.75 319.09 16.03 0.88
1 red-coat.png 0.5 (67, 36, 185) 48.02 301.9 15.15 0.91
1 red-coat.png 0.6 (73, 42, 195) 48.81 306.84 14.48 0.93
1 red-coat.png 0.7 (81, 49, 206) 54.59 343.2 15.24 0.94
Best match:  0.6 14.48
5.07960786001457 -7.288952323610133
Input: D:\Face\lips\images\2.jpg
--------------------
Face is Horizontally inclined by 3.52 degree
Face is Vertically inclined by 9.46 degree
--------------------
2 red-coat.png 0.2 (57, 37, 177) 52.35 329.15 12.83 0.72
2 red-coat.png 0.3 (71, 57, 202) 57.97 364.43 8.42 0.81
2 red-coat.png 0.4 (84, 74, 225) 76.8 482.87 10.2 0.88
2 red-coat.png 0.5 (97, 92, 249) 103.18 648.68 14.79 0.93
2 red-coat.png 0.6 (111, 110, 254) 126.18 793.28 18.54 0.96
2 red-coat.png 0.7 (123, 127, 253) 145.92 917.4 21.56 0.98
Best match:  0.3 8.42
1.0050860052541812 -7.765166018425333
Input: D:\Face\lips\images\3.jpg


15 red-coat.png 0.6 (101, 73, 202) 85.91 540.09 22.95 0.91
15 red-coat.png 0.7 (115, 87, 215) 104.06 654.24 25.55 0.93
Best match:  0.4 20.6
7.399594659887111 -7.037940763184669
Input: D:\Face\lips\images\16.jpg
--------------------
Face is Horizontally inclined by 0.85 degree
Face is Vertically inclined by 0.7 degree
--------------------
16 red-coat.png 0.2 (49, 31, 170) 54.63 343.43 13.72 0.78
16 red-coat.png 0.3 (60, 45, 190) 49.65 312.14 8.36 0.85
16 red-coat.png 0.4 (70, 59, 210) 57.8 363.39 6.18 0.9
16 red-coat.png 0.5 (80, 73, 230) 74.89 470.85 8.94 0.93
16 red-coat.png 0.6 (90, 87, 249) 95.95 603.22 13.25 0.95
16 red-coat.png 0.7 (99, 103, 253) 114.6 720.48 16.09 0.96
Best match:  0.4 6.18
4.537772507906652 -6.241914347415048
Input: D:\Face\lips\images\17.jpg
--------------------
Face is Horizontally inclined by 3.99 degree
Face is Vertically inclined by 8.53 degree
--------------------
17 red-coat.png 0.2 (66, 38, 167) 62.57 393.37 20.64 0.69
17 red-coat.png 0.3 (85, 58, 186) 

Best match:  0.4 22.22
6.170175095029609 1.123302714075429
Input: D:\Face\lips\images\30.jpg
--------------------
Face is Horizontally inclined by 12.97 degree
Face is Vertically inclined by 4.09 degree
--------------------
30 red-coat.png 0.2 (56, 30, 169) 55.24 347.26 16.45 0.78
30 red-coat.png 0.3 (70, 43, 188) 52.01 326.98 14.58 0.84
30 red-coat.png 0.4 (83, 57, 207) 62.04 390.04 14.66 0.89
30 red-coat.png 0.5 (73, 44, 201) 47.9 301.12 12.52 0.92
30 red-coat.png 0.6 (109, 84, 245) 101.66 639.11 19.9 0.94
30 red-coat.png 0.7 (121, 96, 253) 120.02 754.59 23.33 0.95
Best match:  0.5 12.52
-3.5137502606049664 -25.312883801874136


In [306]:
import pandas as pd

results = pd.DataFrame({'imgs': imgs, 'palette': palette, 'thres': thres, 
                        'Hor_angle': Hor_angle, 'Ver_angle': Ver_angle, 
                        'left_eye_angle': left_eye_angle, 'Right_eye_angle': Right_eye_angle,
                        'left_eye_Hi': left_eye_Hi, 'Right_eye_Hi': Right_eye_Hi, 'Lashes': Lashes})
results

,imgs,palette,thres,Hor_angle,Ver_angle,left_eye_angle,Right_eye_angle,left_eye_Hi,Right_eye_Hi,Lashes
0,D:\Face\lips\images\1.jpg,red-coat.png,0.6,1.01,0.52,5.08,7.29,32,33,Yes
1,D:\Face\lips\images\2.jpg,red-coat.png,0.3,3.52,9.46,1.01,7.77,19,19,Yes
2,D:\Face\lips\images\3.jpg,red-coat.png,0.3,0.19,0.97,2.41,2.98,31,31,Yes
3,D:\Face\lips\images\4.jpg,red-coat.png,0.4,11.74,26.84,17.82,3.69,30,30,No
4,D:\Face\lips\images\5.jpg,red-coat.png,0.4,0.00,0.91,5.60,5.39,20,20,Yes
5,D:\Face\lips\images\6.jpg,red-coat.png,0.4,1.36,0.78,2.29,0.00,16,16,Yes
6,D:\Face\lips\images\7.jpg,red-coat.png,0.4,1.00,0.73,6.58,4.24,22,22,Yes
7,D:\Face\lips\images\8.jpg,red-coat.png,0.6,1.05,0.66,2.60,4.33,23,23,Yes
8,D:\Face\lips\images\9.jpg,red-coat.png,0.3,4.53,5.51,2.35,7.03,20,19,Yes
9,D:\Face\lips\images\10.jpg,red-coat.png,0.5,5.13,13.37,8.50,0.00,29,28,No


In [256]:
math.sqrt(((ri_ri[0]-ri_le[0])**2)+((ri_ri[1]-ri_le[1])**2))

0.0

In [263]:
ri_ri, ri_le, ri_up, ri_do

([148, 317], [148, 317], [148, 317], [148, 317])

In [100]:
image = cv2.imread(f'edge{nu}.jpg')
cv2.fillPoly(image, points1, (255))
cv2.imwrite(f'edge{nu}1.jpg', image)

True

In [24]:
import cv2
import numpy as np

cap = cv2.VideoCapture(0)


frame = cv2.imread('temp2.jpg')

hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

lower_red = np.array([30,150,50])
upper_red = np.array([255,255,180])

mask = cv2.inRange(hsv, lower_red, upper_red)
res = cv2.bitwise_and(frame,frame, mask= mask)

edges = cv2.Canny(frame,50,100)
cv2.imwrite('edge.jpg', edges)

True

In [56]:
image = cv2.imread(f'edge{nu}.jpg')
cv2.fillPoly(image, points1, (255))
cv2.imwrite(f'edge{nu}1.jpg', image)

True

In [359]:
import cv2
import mediapipe as mp
import time
import numpy as np
from PIL import Image, ImageDraw, ImageFilter
import image_similarity_measures
from image_similarity_measures.quality_metrics import ssim
from colormath.color_objects import sRGBColor, LabColor
from colormath.color_conversions import convert_color
from colormath.color_diff import delta_e_cie2000
import sys, os
import scipy
import scipy.misc
import scipy.cluster
import binascii
import struct

#cap = cv2.VideoCapture("D:\\Face\\MediaPipe\\output_25fps.mp4") #"Videos/1.mp4"
pTime = 0

def get_dominant_colour(img_path):
    try:
        NUM_CLUSTERS = 5
        im = Image.open(img_path)
        #im = im.resize((150, 150))      # optional, to reduce time
        ar = np.asarray(im)
        shape = ar.shape
        ar = ar.reshape(scipy.product(shape[:2]), shape[2]).astype(float)
        codes, dist = scipy.cluster.vq.kmeans(ar, NUM_CLUSTERS)
        vecs, dist = scipy.cluster.vq.vq(ar, codes)         # assign codes
        counts, bins = scipy.histogram(vecs, len(codes))    # count occurrences    
        index_max = scipy.argmax(counts)                    # find most frequent
        peak = codes[index_max]
        colour = binascii.hexlify(bytearray(int(c) for c in peak)).decode('ascii')
        r = int(peak[0])
        g = int(peak[1])
        b = int(peak[2])
        bgr = (b, g, r)
        return bgr
    except:
        pass
    return (-1,-1,-1)

def compare_color(bgr1, bgr2):
    # Red Color
    color1_rgb = sRGBColor(bgr1[2],bgr1[1], bgr1[0]);

    # Blue Color
    color2_rgb = sRGBColor(bgr2[2],bgr2[1], bgr2[0]);

    # Convert from RGB to Lab Color Space
    color1_lab = convert_color(color1_rgb, LabColor);

    # Convert from RGB to Lab Color Space
    color2_lab = convert_color(color2_rgb, LabColor);

    # Find the color difference
    delta_e = delta_e_cie2000(color1_lab, color2_lab);

#     print ("The difference between the 2 color = ", delta_e)
    return round(delta_e, 2)
    
def compare_SSIM(img1,img2):
#     print(img1,img2)
    in_img1 = cv2.imread(img1)
    in_img2 = cv2.imread(img2)

    img1_outputimage = 'temp1.jpg'
    img2_outputimage = 'temp2.jpg'

    temp1_image = cv2.resize(in_img1, (100, 50), interpolation=cv2.INTER_NEAREST)
    cv2.imwrite(img1_outputimage, temp1_image)

    temp2_image = cv2.resize(in_img2, (100, 50), interpolation=cv2.INTER_NEAREST)
    cv2.imwrite(img2_outputimage, temp2_image)

    out_ssim = ssim(temp1_image, temp2_image) # Structural Similar Index Measure (SSIM)
#     print('SSIM : ', out_ssim)
    return round(out_ssim,2)

def AngleBtw2Points(pointA, pointB):
    changeInX = pointB[0] - pointA[0]
    changeInY = pointB[1] - pointA[1]
    return math.degrees(math.atan2(changeInY,changeInX))

imgs, palette, thres, Hor_angle, Ver_angle, left_eye_angle, left_eye_Hi, Right_eye_angle, Right_eye_Hi, Lashes = [], [], [], [], [], [], [], [], [], []

images = 30
for nu in range(1,images+1):
# for nu in [5]:
    imgPath = f"D:\\Face\lips\\images\\{nu}.jpg"
    print("Input:", imgPath)
    paletteImages = "red-coat.png,ruby-rush.png,red-rust.png"
    temp_image = r"temp.jpg"
    eye_le = Image.open('lashes.png')
    eye_ri = Image.open('lashes1.png')
#     eye_ri = eye_up.transpose(Image.FLIP_LEFT_RIGHT)
    
    mpDraw = mp.solutions.drawing_utils
    mpFaceMesh = mp.solutions.face_mesh
    faceMesh = mpFaceMesh.FaceMesh(max_num_faces=2)
    drawSpec = mpDraw.DrawingSpec(thickness=1, circle_radius=2)

    face = [10, 338, 297, 332, 284, 251, 389, 356, 454, 323, 361, 288, 397, 365, 379, 378, 400, 377, 152, 
            148, 176, 149, 150, 136, 172, 58, 132, 93, 234, 127, 162, 21, 54, 103, 67, 109, 10, 168,
            133, 173, 157, 158, 159, 160, 161, 246, 22, 7, 163, 144, 145, 153, 154, 155, 133, 168,
            362, 398, 384, 385, 386, 387, 388, 466, 263, 249, 390, 373, 374, 380, 381, 382, 362, 168,
            0, 267, 269, 270, 409, 291, 375, 321, 405, 314, 17, 84, 181, 91, 146, 61, 185, 40, 39, 37, 0, 168]
    
    face_up = [10, 338, 297, 332, 284, 103, 67, 109, 67]
    
    lips = [61, 185, 40, 39, 37, 0, 267, 269, 270, 409, 291,
            375, 321, 405, 314, 17, 84, 181, 91, 146, 61,
            78, 191, 80, 81, 82, 13, 312, 311, 310, 415, 308,
            324, 318, 402, 317, 14, 87, 178, 88, 95, 78]

    lips_up = [61, 185, 40, 39, 37, 0, 267, 269, 270, 409, 291]
    lips_down = [375, 321, 405, 314, 17, 84, 181, 91, 146, 61]


    l_eye = [33, 161, 160, 159, 158, 157, 173, 133, 155, 154, 153, 145, 144, 163]
    r_eye = [362, 398, 384, 385, 386, 387, 388, 466, 263, 249, 390, 373, 374, 380, 381, 382]
    
    #while True:
        #success, img = cap.read()
    img = cv2.imread(imgPath)
    ih, iw, ic = img.shape
    overlay = img.copy()

    frame = cv2.imread(imgPath)
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    lower_red = np.array([30,150,50])
    upper_red = np.array([255,255,180])
    mask = cv2.inRange(hsv, lower_red, upper_red)
    res = cv2.bitwise_and(frame,frame, mask= mask)
    edges = cv2.Canny(frame,50,100)
    cv2.imwrite(f'edge{nu}.jpg', edges)
    edg = cv2.imread(f'edge{nu}.jpg')

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    paletteImageList = paletteImages.split(',')
    index =1
    # print(paletteImageList)
    height = overlay.shape[0]
    width = overlay.shape[1]
    empty_mask = np.zeros((height, width), dtype=np.uint8)

    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # imgRGB = cv2.cvtColor(edg, cv2.COLOR_BGR2RGB)

    results = faceMesh.process(imgRGB)
    if results.multi_face_landmarks:
    #print(results.multi_face_landmarks)
        for faceLms in results.multi_face_landmarks:
            mpDraw.draw_landmarks(img, faceLms, mpFaceMesh.FACE_CONNECTIONS, drawSpec,drawSpec)

            points = []
            le_pts = []
            re_pts = []
            fpoints = []
            for id,lm in enumerate(lips):
                xx = int(faceLms.landmark[lm].x*iw)
                yy = int(faceLms.landmark[lm].y*ih) 
                points.append((xx, yy)) 
                y_flag = yy
                if lm in lips_up:
                    points[id] = (xx, yy-yy*0.001)

                elif lm in lips_down:
                    points[id] = (xx, yy+yy*0.001)
                    
            for id,lm in enumerate(face):
                xx = int(faceLms.landmark[lm].x*iw)
                yy = int(faceLms.landmark[lm].y*ih) 
                fpoints.append((xx, yy)) 
                y_flag = yy
                if lm in face_up:
                    for ii in range(100):
#                         print(edg[xx,yy-ii])
                        if edg[yy-ii,xx][0] != 0 and edg[yy-ii,xx][1] != 0 and edg[yy-ii,xx][2] != 0:
                            fpoints[id] = (xx, yy-ii)
                            print(xx,yy,ii)
                            
                            break
                    
            for id,lm in enumerate(l_eye):
                if lm in l_eye:
                    xx = int(faceLms.landmark[lm].x*iw)
                    yy = int(faceLms.landmark[lm].y*ih)                
                    le_pts.append((xx, yy))  
                    if lm == 33:
                        le_le = [xx, yy]
                    if lm == 133:
                        le_ri = [xx, yy]                    
                    if lm == 159:
                        le_up = [xx, yy]                    
                    if lm == 145:
                        le_do = [xx, yy]
                        
            for id,lm in enumerate(r_eye):
                if lm in r_eye:
                    xx = int(faceLms.landmark[lm].x*iw)
                    yy = int(faceLms.landmark[lm].y*ih) 
                    if lm == 362:
                        ri_le = [xx, yy]
                    if lm == 263:
                        ri_ri = [xx, yy]                    
                    if lm == 386:
                        ri_up = [xx, yy]                    
                    if lm == 374:
                        ri_do = [xx, yy]                        
                        
            h1 = [int(faceLms.landmark[33].x*iw), int(faceLms.landmark[33].y*ih)]
            h2 = [int(faceLms.landmark[263].x*iw), int(faceLms.landmark[263].y*ih)]
            v1 = [int(faceLms.landmark[5].x*iw), int(faceLms.landmark[5].y*ih)]
            v2 = [int(faceLms.landmark[0].x*iw), int(faceLms.landmark[0].y*ih)]
            lpt1 = [int(faceLms.landmark[70].x*iw), int(faceLms.landmark[70].y*ih)]
    print('-'*20)
    print("Face is Horizontally inclined by", abs(round(AngleBtw2Points(h1, h2),2)), 'degree')
    print("Face is Vertically inclined by", abs(round(90-AngleBtw2Points(v1, v2),2)), 'degree')
    print('-'*20)

    # points.append(points[0])    
    points = np.reshape(points, (-1, 1, 2))  
    points1 = np.int32([points])
    cv2.fillPoly(img, points1, (0, 0, 255), 8)         

    for paletteImage in paletteImageList:
        
        im_final = apply_makeup('cool-ivory.png', imgPath, fpoints, empty_mask)
        im_final.save(f"Out_{paletteImage}")
        
        imgPath = f"Out_{paletteImage}"
                                
        imgs.append(imgPath)
        palette.append(paletteImage)    
            
    #     print(paletteImage)
        #imgOutPath = outputFolder + str(index) + '.jpg'
        bgr_p = get_dominant_colour(paletteImage)
    #     print(bgr_p)

        #cv2.imwrite('raw.jpg',img)

        cv2.fillPoly(empty_mask, points1, (255))
        res = cv2.bitwise_and(overlay,overlay,mask = empty_mask)

        background = cv2.imread(imgPath)
        overlay = cv2.imread(paletteImage)
        overlay= cv2.resize(overlay, (background.shape[1], background.shape[0]))

        rect = cv2.boundingRect(points.astype(np.int))# returns (x,y,w,h) of the rect
        cropped = background[rect[1]-20: rect[1] + rect[3]+20, rect[0]-10: rect[0] + rect[2]+10]
        cv2.imwrite('temp2.jpg', cropped)

        sharpen_filter = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
        background = cv2.filter2D(background, -1, sharpen_filter)

        color_control = []
        color_SSIM = []
        color_Labspace = []
        for i in range(2,8,1):
            added_image = cv2.addWeighted(background,i/10,overlay,0.6,0)
            time.sleep(1)
            cv2.imwrite(temp_image, added_image)

            orig = cv2.imread(temp_image)
            rect = cv2.boundingRect(points.astype(np.int)) # returns (x,y,w,h) of the rect
    #         print(rect[1], rect[1] + rect[3], rect[0], rect[0] + rect[2])
            cropped = orig[rect[1]-20: rect[1] + rect[3]+20, rect[0]-10: rect[0] + rect[2]+10]
            cv2.imwrite('temp1.jpg', cropped)

            bgr = get_dominant_colour('temp1.jpg')

            d=((bgr[0]-bgr_p[0])**2+(bgr[1]-bgr_p[1])**2+(bgr[2]-bgr_p[2])**2)**0.5
            p=d*100/((255)^2+(255)^2+(255)^2)**0.5

            color_control.append(i/10)
            color_SSIM.append(round(compare_SSIM('temp2.jpg','temp1.jpg'),2))
            color_Labspace.append(round(compare_color(bgr_p, bgr),2))
            print(nu, paletteImage, i/10 ,bgr, round(d,2), round(p,2), compare_color(bgr_p, bgr),compare_SSIM('temp2.jpg','temp1.jpg'))

        color_control1 = []
        color_SSIM1 = []
        color_Labspace1 = []
        for i, val in enumerate(color_SSIM):
            if val > 0.8:
                color_control1.append(color_control[i])
                color_SSIM1.append(color_SSIM[i])
                color_Labspace1.append(color_Labspace[i])
#                 print(color_Labspace1)

        added_image = cv2.addWeighted(background,color_control1[color_Labspace1.index(min(color_Labspace1))],overlay,0.6,0)
        cv2.imwrite(temp_image, added_image)
        thresh_val = color_control1[color_Labspace1.index(min(color_Labspace1))]
        print('Best match: ', thresh_val, min(color_Labspace1))

        thres.append(thresh_val)
        
        im2 = Image.open(imgPath)
        im1 = Image.open(temp_image)

        height = im2.size[1]
        width = im2.size[0]

        empty_mask = np.zeros((height, width), dtype=np.uint8)
        cv2.fillPoly(empty_mask, points1, (255,255,255))
        cv2.imwrite('mask.jpg', empty_mask)


        mask = Image.open('mask.jpg').convert('L').resize(im1.size)
        mask_blur = mask.filter(ImageFilter.GaussianBlur(3))
        im_final = Image.composite(im1, im2, mask_blur)
        
        #Left eye
        # Calculate Euclidean distance
        eye1_w = math.sqrt(((le_ri[0]-le_le[0])**2)+((le_ri[1]-le_le[1])**2))
        eye1_h = math.sqrt(((le_do[0]-le_up[0])**2)+((le_do[1]-le_up[1])**2))
        rot1 = AngleBtw2Points(le_le, le_ri)

        le_x_resize = int(eye1_w*1.4)
        le_y_resize = int(eye1_h*2.4)

        eye_le = eye_le.resize((le_x_resize,le_y_resize))
        eye_le = eye_le.rotate(rot1*-1+4, expand=True)

        #Right_eye
        eye2_w = math.sqrt(((ri_ri[0]-ri_le[0])**2)+((ri_ri[1]-ri_le[1])**2))
        eye2_h = math.sqrt(((ri_do[0]-ri_up[0])**2)+((ri_do[1]-ri_up[1])**2))
        rot2 = AngleBtw2Points(ri_le, ri_ri)

        ri_x_resize = int(eye2_w*1.4)
        ri_y_resize = int(eye2_h*2.4)
        
        eye_ri = eye_ri.resize((ri_x_resize,ri_y_resize))
        eye_ri = eye_ri.rotate(rot2*-1-4, expand=True)        
        print(rot1,rot2)
        
        #lashes
        width = 800
        height = 519
        Ymargin_le = 320*le_y_resize/height
        Xmargin_le = 220*le_x_resize/width
        Ymargin_ri = 370/(height/ri_y_resize)
        Xmargin_ri = -10/(width/ri_x_resize)

        im_final.paste(eye_le, (int(le_le[0]-Xmargin_le),int(le_le[1]-Ymargin_le)), mask = eye_le)
        im_final.paste(eye_ri, (int(ri_le[0]-Xmargin_ri),int(ri_le[1]-Ymargin_ri)), mask = eye_ri)
#         im_final.save(f"Out_{paletteImage}")
        

        im_final.save(f"{nu}_Out_{paletteImage}")

        Hor_angle.append(abs(round(AngleBtw2Points(h1, h2),2)))
        Ver_angle.append(abs(round(90-AngleBtw2Points(v1, v2),2)))
        left_eye_angle.append(abs(round(rot1,2)))
        left_eye_Hi.append(round(eye1_h))
        Right_eye_angle.append(abs(round(rot2,2)))
        Right_eye_Hi.append(round(eye2_h))
        
        if abs(rot1) > 8 or abs(rot2) > 8 or eye1_h < 10 or eye2_h < 10:
            Lashes.append('No')
        else:
            Lashes.append('Yes')
        
        index = index +1
        break
#     os.remove(f'edge{nu}.jpg')

os.remove('temp.jpg')
os.remove('temp1.jpg')
os.remove('temp2.jpg')


Input: D:\Face\lips\images\1.jpg
237 149 38
297 149 15
390 171 37
415 196 6
79 175 17
176 150 1
237 149 38
--------------------
Face is Horizontally inclined by 1.01 degree
Face is Vertically inclined by 0.52 degree
--------------------


C:\Users\css120809\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: DeprecationWarning: scipy.product is deprecated and will be removed in SciPy 2.0.0, use numpy.product instead
C:\Users\css120809\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: DeprecationWarning: scipy.histogram is deprecated and will be removed in SciPy 2.0.0, use numpy.histogram instead
C:\Users\css120809\Anaconda3\lib\site-packages\ipykernel_launcher.py:32: DeprecationWarning: scipy.argmax is deprecated and will be removed in SciPy 2.0.0, use numpy.argmax instead


1 red-coat.png 0.2 (51, 25, 164) 57.58 362.03 17.17 0.76
1 red-coat.png 0.3 (61, 37, 181) 49.93 313.91 13.2 0.83
1 red-coat.png 0.4 (71, 48, 197) 51.87 326.07 11.56 0.88
1 red-coat.png 0.5 (81, 59, 213) 61.98 389.64 12.01 0.91
1 red-coat.png 0.6 (91, 71, 228) 77.82 489.25 13.9 0.93
1 red-coat.png 0.7 (102, 83, 241) 95.99 603.48 16.98 0.94
Best match:  0.4 11.56
5.07960786001457 -7.288952323610133
Input: D:\Face\lips\images\2.jpg
170 211 28
213 208 42
253 211 0
293 220 5
324 238 1
78 239 1
101 224 9
131 215 8
170 211 28
--------------------
Face is Horizontally inclined by 3.52 degree
Face is Vertically inclined by 9.46 degree
--------------------
2 red-coat.png 0.2 (55, 37, 178) 51.38 323.03 11.76 0.72
2 red-coat.png 0.3 (68, 55, 202) 55.18 346.92 7.16 0.81
2 red-coat.png 0.4 (81, 72, 225) 73.74 463.62 9.13 0.88
2 red-coat.png 0.5 (98, 96, 253) 108.24 680.47 15.43 0.93
2 red-coat.png 0.6 (107, 108, 252) 122.07 767.45 17.51 0.96
2 red-coat.png 0.7 (119, 125, 252) 142.06 893.1 20.7 0.98


13 red-coat.png 0.7 (151, 141, 254) 172.6 1085.15 29.95 0.98
Best match:  0.4 15.9
6.581944655178011 -6.581944655178011
Input: D:\Face\lips\images\14.jpg
217 222 16
247 222 25
274 226 5
299 235 0
316 249 2
141 240 0
163 230 2
187 224 19
217 222 16
--------------------
Face is Horizontally inclined by 1.92 degree
Face is Vertically inclined by 2.82 degree
--------------------
14 red-coat.png 0.2 (67, 44, 180) 56.75 356.81 15.24 0.69
14 red-coat.png 0.3 (80, 61, 201) 65.31 410.63 13.13 0.79
14 red-coat.png 0.4 (103, 86, 229) 96.88 609.09 17.5 0.86
14 red-coat.png 0.5 (116, 103, 250) 121.87 766.21 20.41 0.92
14 red-coat.png 0.6 (132, 123, 253) 147.34 926.34 24.48 0.95
14 red-coat.png 0.7 (148, 142, 253) 171.47 1078.04 28.61 0.98
Best match:  0.4 17.5
1.3019526725788753 -4.864514437760526
Input: D:\Face\lips\images\15.jpg
211 220 29
267 221 3
315 226 28
358 239 0
384 260 1
74 242 14
112 228 37
157 222 1
211 220 29
--------------------
Face is Horizontally inclined by 0.22 degree
Face is Ve

26 red-coat.png 0.2 (59, 38, 174) 55.58 349.42 14.55 0.76
26 red-coat.png 0.3 (73, 56, 195) 60.12 377.95 11.43 0.83
26 red-coat.png 0.4 (87, 74, 218) 77.83 489.29 11.97 0.88
26 red-coat.png 0.5 (101, 92, 239) 102.52 644.53 15.53 0.92
26 red-coat.png 0.6 (115, 109, 253) 127.03 798.64 19.61 0.94
26 red-coat.png 0.7 (129, 127, 253) 148.97 936.59 23.09 0.95
Best match:  0.3 11.43
10.619655276155134 -3.0127875041833403
Input: D:\Face\lips\images\27.jpg
501 304 1
548 308 1
582 317 0
607 334 0
618 360 1
335 325 2
391 309 1
445 304 0
501 304 1
--------------------
Face is Horizontally inclined by 0.23 degree
Face is Vertically inclined by 6.52 degree
--------------------
27 red-coat.png 0.2 (51, 32, 166) 58.59 368.36 15.53 0.75
27 red-coat.png 0.3 (62, 46, 183) 55.02 345.9 11.24 0.82
27 red-coat.png 0.4 (73, 61, 202) 62.26 391.41 8.7 0.88
27 red-coat.png 0.5 (83, 75, 218) 76.97 483.89 9.79 0.91
27 red-coat.png 0.6 (114, 110, 246) 125.61 789.71 19.02 0.94
27 red-coat.png 0.7 (126, 126, 248) 145

In [341]:
def apply_makeup(paletteImage, imgPath, points, empty_mask):
    points = np.reshape(points, (-1, 1, 2))  
    points1 = np.int32([points])
    
    background = cv2.imread(imgPath)
    overlay = background.copy()
    
    bgr_p = get_dominant_colour(paletteImage)
    cv2.fillPoly(empty_mask, points1, (255))
    res = cv2.bitwise_and(overlay,overlay,mask = empty_mask)

    
    overlay = cv2.imread(paletteImage)
    overlay= cv2.resize(overlay, (background.shape[1], background.shape[0]))

    rect = cv2.boundingRect(points.astype(np.int))# returns (x,y,w,h) of the rect
    cropped = background[rect[1]-20: rect[1] + rect[3]+20, rect[0]-10: rect[0] + rect[2]+10]
    cv2.imwrite('temp2.jpg', cropped)

    sharpen_filter = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
    background = cv2.filter2D(background, -1, sharpen_filter)

    added_image = cv2.addWeighted(background,0.6,overlay,0.4,0)
    cv2.imwrite("temp.jpg", added_image)

    im2 = Image.open(imgPath)
    im1 = Image.open("temp.jpg")

    height = im2.size[1]
    width = im2.size[0]

    empty_mask = np.zeros((height, width), dtype=np.uint8)
    cv2.fillPoly(empty_mask, points1, (255,255,255))
    cv2.imwrite('mask.jpg', empty_mask)


    mask = Image.open('mask.jpg').convert('L').resize(im1.size)
    mask_blur = mask.filter(ImageFilter.GaussianBlur(3))
    im_final = Image.composite(im1, im2, mask_blur)
    return im_final

array([0, 0, 0], dtype=uint8)